# Deploying a Model as a Web Service

It's no good being able to create an accurate model if you can't deploy it for use in an application or service. In this notebook, we'll explore the *Azure Machine Learning Service* and the associated *Azure Machine Learning SDK*; which together enable you to train, deploy, and manage machine learning models at scale.

To use Azure Machine Learning, you're going to need an Azure subscription. If you don't already have one, you can sign up for a free trial at https://azure.microsoft.com/Account/Free.

*Note: Azure Machine Learning provides a whole range of functionality to help you through the entire lifecycle of model development, training, evaluation, deployment, and management. We're going to focus on using it to deploy a trained model; but you can use it to do much, much more!*

## A Brief Introduction to Containers
When you access a web site or a software service across the internet, you're probably dimly aware that somewhere, the code for the service is hosted on a *server*. We tend to think of servers as being physical computers, but in recent years there's been a growth in *virtualization* technologies so that a computer can be virtualized in software, and multiple *virtual machines* can be hosted on a single physical server.

Virtual machines (VMs) are useful - in fact, the Azure Data Science Virtual Machine (DSVM) is a good example of a VM that enables you to provision a computer that contains the operating system (OS) and all the software applications you need to work with data and build machine learning models, and then you can delete the VM when you're finished with it so that you only pay for what you use - very cool!

However, it seems wasteful to provision a complete virtual machine, including the full OS and applications, just to host a simple software service - especially if you need to support multiple services, each one consuming its own VM. *Containers* are an evolutionary step beyond VMs. They contain only the OS components that are required for the specific software service they need to host. This makes them very small compared to full VMs, which in turn means that they're portable, and quick to deploy and start up.

Containers themselves are hosted in a container environment that provides all the common services and OS functionality they require. During development, this environment is often a locally installed system called *Docker*. When hosting a service in the cloud however, you can use container services such as *Azure Container Instances* (ACI), which is useful for lightweight hosting and testing of containerized services; or *Azure Kubernetes Services*, which provides a scalable and highly-available environment for managing clusters of containers, based on the industry standard *Kubernetes* container hosting platform.

In the rest of this notebook, we'll examine how you can use Azure Machine Learning Services to prepare a container image for your machine learning model, and deploy your model as a containerized web service that can be consumed by other applications that connect to it over an HTTP REST endpoint.

## Train a classification model
Let's start by training a simple classification model so that we have something to deploy.
If you've viewed the previous sample notebooks in this hack, this should be pretty familiar - we're going to use Keras to train a simple shape classifier.

First, we'll import the libraries we're going to need to train and validate a model using Keras.

In [ ]:
import sys
! {sys.executable} -m pip install --upgrade keras

import numpy as np
import matplotlib.pyplot as plt
import tensorflow, keras
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

print("Ready to train a model using Keras %s and TensorFlow %s" % (keras.__version__, tensorflow.__version__))

Next we'll generate some images of geometric shapes with which to train and validate the model.

In [ ]:
# Function to create a random image (of a square, circle, or triangle)
def create_image (size, shape):
    from random import randint
    import numpy as np
    from PIL import Image, ImageDraw
    
    xy1 = randint(10,40)
    xy2 = randint(60,100)
    col = (randint(0,200), randint(0,200), randint(0,200))

    img = Image.new("RGB", size, (255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    if shape == 'circle':
        draw.ellipse([(xy1,xy1), (xy2,xy2)], fill=col)
    elif shape == 'square':
        draw.rectangle([(xy1,xy1), (xy2,xy2)], fill=col)
    else: # triangle
        draw.polygon([(xy1,xy1), (xy2,xy2), (xy2,xy1)], fill=col)
    del draw
    
    return np.array(img)

# function to create a dataset of images
def generate_image_data (classes, size, cases, img_dir):
    import os, shutil
    from PIL import Image
    
    if os.path.exists(img_dir):
        replace_folder = input("Image folder already exists. Enter Y to replace it (this can take a while!). \n")
        if replace_folder == "Y":
            print("Deleting old images...")
            shutil.rmtree(img_dir)
        else:
            return # Quit - no need to replace existing images
    os.makedirs(img_dir)
    print("Generating new images...")
    i = 0
    while(i < (cases - 1) / len(classes)):
        if (i%25 == 0):
            print("Progress:{:.0%}".format((i*len(classes))/cases))
        i += 1
        for classname in classes:
            img = Image.fromarray(create_image(size, classname))
            saveFolder = os.path.join(img_dir,classname)
            if not os.path.exists(saveFolder):
                os.makedirs(saveFolder)
            imgFileName = os.path.join(saveFolder, classname + str(i) + '.jpg')
            try:
                img.save(imgFileName)
            except:
                try:
                    # Retry (resource constraints in Azure notebooks can cause occassional disk access errors)
                    img.save(imgFileName)
                except:
                    # We gave it a shot - time to move on with our lives
                    print("Error saving image", imgFileName)
            
# Our classes will be circles, squares, and triangles
classnames = ['circle', 'square', 'triangle']

# All images will be 128x128 pixels
img_size = (128,128)

# We'll store the images in a folder named 'shapes'
data_folder = 'shapes'

# Generate 1200 random images.
generate_image_data(classnames, img_size, 1200, data_folder)

batch_size = 30

print("Loading Data from ...%s folder!" % data_folder)
datagen = ImageDataGenerator(rescale=1./255, # normalize pixel values
                             validation_split=0.3) # hold back 30% of the images for validation

print("Preparing training dataset...")
train_generator = datagen.flow_from_directory(
    data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator = datagen.flow_from_directory(
    data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classes = sorted(train_generator.class_indices.keys())
print("Data loaded. \n Class names: ", classes)

Now we'll define and train the model.

In [ ]:
# Define a CNN classifier network
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers

# Define the model as a sequence of layers
model = Sequential()

# The input layer accepts an image and applies a convolution that uses 32 6x6 filters and a rectified linear unit activation function
model.add(Conv2D(32, (6, 6), input_shape=train_generator.image_shape, activation='relu'))

# Next we'll add a max pooling layer with a 2x2 patch
model.add(MaxPooling2D(pool_size=(2,2)))

# We can add as many layers as we think necessary - here we'll add another convolution, max pooling, and dropout layer
model.add(Conv2D(32, (6, 6), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# A dropout layer randomly drops some nodes to reduce inter-dependencies (which can cause over-fitting)
model.add(Dropout(0.2))

# Now we'll flatten the feature maps and generate an output layer with a predicted probability for each class
model.add(Flatten())
model.add(Dense(train_generator.num_classes, activation='softmax'))

# With the layers defined, we can now compile the model for categorical (multi-class) classification
opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Train the model over 5 epochs using 30-image batches and using the validation holdout dataset for validation
num_epochs = 5
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)

## Save and test the model locally
OK, so we now have a trained shape classification model. Let's save it as a local file (well, local to the Azure Notebooks library anyway), and then load and test it; just to satisfy ourselves that it works:

In [ ]:
def predict_image(classifier, image_array):
    import numpy as np
    
    # We need to format the input to match the training data
    # The generator loaded the values as floating point numbers
    # and normalized the pixel values, so...
    imgfeatures = image_array.astype('float32')
    imgfeatures /= 255
    
    # These are the classes our model can predict
    classnames = ['circle', 'square', 'triangle']
    
    # Predict the class of each input image
    predictions = classifier.predict(imgfeatures)
    
    predicted_classes = []
    for prediction in predictions:
        # The prediction for each image is the probability for each class, e.g. [0.8, 0.1, 0.2]
        # So get the index of the highest probability
        class_idx = np.argmax(prediction)
        # And append the corresponding class name to the results
        predicted_classes.append(classnames[int(class_idx)])
    # Return the predictions as a JSON
    return predicted_classes


from keras.models import load_model
from random import randint

modelFileName = 'shape-classifier.h5'

model.save(modelFileName) # saves the trained model
del model  # deletes the existing model variable

model = load_model(modelFileName) # loads the saved model

# Create a random test image
img = create_image ((128,128), classes[randint(0, len(classes)-1)])
plt.imshow(img)

# Create an array of (1) images to match the expected input format
img_array = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

# get the predicted clases
predicted_classes = predict_image(model, img_array)

# Display the prediction for the first image (we only submitted one!)
print(predicted_classes[0])

It looks as though we have a working model. Now we're ready to use Azure Machine Learning to deploy it as a web service.

## Create an Azure Machine Learning workspace

To use Azure Machine Learning, you'll need to create a workspace in your Azure subscription.

Your Azure subscription is identified by a subscription ID. To find this:
1. Sign into the Azure portal at https://portal.azure.com.
2. On the menu tab on the left, click &#128273; **Subscriptions**.
3. View the list of your subscriptions and copy the ID for the subscription you want to use.
4. Paste the subscription ID into the code below, and then run the cell to set the variable - you will use it later.

In [ ]:
# Replace YOUR_SUBSCRIPTION_ID in the following variable assignment:
SUBSCRIPTION_ID = 'YOUR_SUBSCRIPTION_ID'

To deploy the model file as a web service, we'll use the Azure Machine Learning SDK.

> Note: the Azure Machine Learning SDK is installed by default in Azure Notebooks and the Azure Data Science Virtual Machine, but you may want to ensure that it's upgraded to the latest version. If you're using your own Python environment, you'll need to install it using the instructions in the [Azure Machine Learning documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python)*

In [ ]:
import sys
! {sys.executable} -m pip install azureml-sdk --upgrade

import azureml.core
print(azureml.core.VERSION)

To manage the deployment, we need an Azure ML workspace. Create one in your Azure subscription by running the following cell. You'll be prompted to authenticate by entering a code at a given URL, so just click the link that's displayed and enter the specified code.

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name='my_aml_workspace', # or another name of your choosing
                      subscription_id=SUBSCRIPTION_ID,
                      resource_group='aml_resources', # or another name of your choosing
                      create_resource_group=True,
                      location='eastus2' # or other supported Azure region
                     )

Now that you have a workspace, you can save the configuration so you can reconnect to it later.

In [ ]:
from azureml.core import Workspace

# Save the workspace config
ws.write_config()

# Reconnect to the workspace (if you're not already signed in, you'll be prompted to authenticate with a code as before)
ws = Workspace.from_config()

## Create a *scoring* file
Your web service will need some Python code to load the input data, get the model, and generate and return a prediction. We'll save this code in a *scoring* file that will be deployed to the web service:

In [ ]:
%%writefile score_keras.py
# scoring script used by service to load model and generate prediction
import json
import numpy as np
import os
from keras.models import load_model
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('shape-classifier.h5')
    model = load_model(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data - the image(s) to be classified.
    data = np.array(json.loads(raw_data)['data'])
    
    # Pre-process the images
    imgfeatures = data.astype('float32')
    imgfeatures /= 255

    # Get a prediction from the model
    predictions = model.predict(imgfeatures)
    # get thge classname for the highest probability prediction for each input
    classnames = ['circle', 'square', 'triangle']
    predicted_classes = []
    for prediction in predictions:
        class_idx = np.argmax(prediction)
        predicted_classes.append(classnames[int(class_idx)])
    # Return the predictions as a JSON
    return json.dumps(predicted_classes)



## Create an *environment* file
The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires **scikit-learn**, so we'll create a .yml file that tells the container host to install this into the environment.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("keras")

env_file = "env_keras.yml"

with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

## Define a container image
We're going to deploy the web service as a container, so we need to define a container image that includes our scoring file and denvironment dependencies.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score_keras.py",
                                                  runtime = "python",
                                                  conda_file = "env_keras.yml",
                                                  description = "Container image for shape classification",
                                                  tags = {"data": "shapes", "type": "classification"}
                                                 )
print(image_config.description)

## Define the web service deployment configuration
We're going to deploy the containerized web service in the Azure Container Instance (ACI) service, so we need to specify the deployment configuration.

In [ ]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "shapes", "type": "classification"},
                                               description = 'shape classification service')
print(aci_config.description)

## Deploy the web service 
Now we're ready to deploy. We'll deploy the container a service named **aci-shape-svc**.
The deployment process includes the following steps:
1. Register the model file in the Azure Machine Learning service (this also uploads the local model file to your Azure Machine Learning service so it can be deployed to a container)
2. Create a container image for the web service, based on the configuration specified previously. This image will be used to instantiate the service.
3. Create a service by deploying the container image (in this case to ACI - other hosts are available!)
4. Verify the status of the deployed service.

This will take some time. When deployment has completed successfully, you'll see a status of **Healthy**.

In [ ]:
from azureml.core.webservice import Webservice

service_name = 'aci-shape-keras-svc'
service = Webservice.deploy(deployment_config = aci_config,
                                image_config = image_config,
                                model_paths = ['shape-classifier.h5'],
                                name = service_name,
                                workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

## Use the web service
With the service deployed, now we can test it by using it to predict the shape of a new image.

In [ ]:
import json
from random import randint

# Create a random test image
img = create_image ((128,128), classes[randint(0, len(classes)-1)])
plt.imshow(img)

# Modify the image data to create an array of 1 image, matching the format of the training features
input_array = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

# Convert the array to JSON format
input_json = json.dumps({"data": input_array.tolist()})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
classname = json.loads(predictions)[0]
print('The image is a', classname)

You can also send a batch of images to the service, and get back a prediction for each one.

In [ ]:
import json
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline

# Create three random test images
fig = plt.figure(figsize=(6, 6))
images = []
i = 0
while(i < 3):  
    # Create a new image
    img = create_image((128,128), classes[randint(0, len(classes)-1)])
    # Plot the image
    a=fig.add_subplot(1,3,i + 1)
    imgplot = plt.imshow(img)
    # Add the image to an array to be submitted as a batch
    images.append(img.tolist())
    i += 1

# Convert the array to JSON format
input_json = json.dumps({"data": images})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes
print(json.loads(predictions))

### Using the Web Service from Other Applications
The code above uses the Azure ML SDK to connect to the containerized web service and use it to generate predictions from your image classification model. In production, the model is likely to be consumed by business applications that make HTTP requests to the web service.

Let's determine the URL to which these applications must submit their requests:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Now that we know the endpoint URI, an application can simply make an HTTP request, sending the image data in JSON (or binary) format, and receive back the predicted class(es).

In [ ]:
import requests
import requests
import json
from random import randint

# Create a random test image
img = create_image ((128,128), classes[randint(0, len(classes)-1)])
plt.imshow(img)

# Create an array of (1) images to match the expected input format
image_array = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": image_array.tolist()})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
print(json.loads(predictions.content))

## Deleting the Service

When we're finished with the service, we can delete it to avoid incurring charges.

In [ ]:
service.delete()
print("Service deleted.")

And if you're finished with the workspace, you can delete that too

In [ ]:
rg = ws.resource_group
ws.delete()
print("Workspace deleted. You should delete the '%s' resource group in your Azure subscription." % rg)

## Learn more
Take a look at the Azure Machine Learning documentation at https://docs.microsoft.com/en-us/azure/machine-learning/service/.